In [1]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

import tensorflow as tf
from tensorflow.keras import layers, Model
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.utils import plot_model

print(tf.__version__)

2024-12-22 14:45:58.990234: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-22 14:45:59.002471: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-22 14:45:59.005481: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-22 14:45:59.015753: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2.17.0


In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


I0000 00:00:1734867962.435943   34081 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1734867962.509371   34081 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1734867962.509461   34081 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [3]:

# Убедимся, что TensorFlow использует GPU
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    try:
        tf.config.experimental.set_memory_growth(physical_devices[0], True)
        # os.environ['TF_GPU_ALLOCATOR']='cuda_malloc_async'
        print("GPU доступен и будет использован для обучения.")
    except RuntimeError as e:
        print(f"Ошибка настройки памяти GPU: {e}")
else:
    print("GPU не обнаружен. Используется CPU.")

GPU доступен и будет использован для обучения.


In [4]:
# Параметры датасета
DATASET_ROOT_DIR = "/mnt/d/"
DATASET_DIR = os.path.join(DATASET_ROOT_DIR, "Agriculture-Vision-2021 2")   # Задайте путь к датасету
TRAIN_DIR = os.path.join(DATASET_DIR, "train")
VAL_DIR = os.path.join(DATASET_DIR, "val")
IMG_SIZE = 512  # Размер входного изображения
BATCH_SIZE = 16
NUM_CLASSES = 9  # Количество классов сегментации

In [5]:
import sys

if not os.path.isdir(DATASET_ROOT_DIR):
    sys.exit("Dataset dir is not exist")

In [6]:
# # Патчинг изображения
# class PatchEmbedding(layers.Layer):
#     def __init__(self, patch_size, embed_dim):
#         super().__init__()
#         self.patch_size = patch_size
#         self.embed_dim = embed_dim

#     def build(self, input_shape):
#         self.projection = layers.Conv2D(
#             filters=self.embed_dim,
#             kernel_size=self.patch_size,
#             strides=self.patch_size,
#             padding="valid"
#         )
#         self.flatten = layers.Reshape((-1, self.embed_dim))

#     def call(self, inputs):
#         x = self.projection(inputs)
#         x = self.flatten(x)
#         return x

In [8]:

# # Модель ViT для семантической сегментации
# class VisionTransformer(Model):
#     def __init__(self, img_size, patch_size, num_classes, embed_dim, depth, num_heads, mlp_dim, dropout_rate=0.1):
#         super().__init__()
#         self.img_size = img_size
#         self.patch_size = patch_size
#         self.num_patches = (img_size // patch_size) ** 2

#         self.patch_embedding = PatchEmbedding(patch_size, embed_dim)
#         self.position_embedding = self.add_weight(
#             shape=(1, self.num_patches, embed_dim),
#             initializer="random_normal",
#             trainable=True
#         )

#         self.transformer_blocks = [
#             TransformerBlock(embed_dim, num_heads, mlp_dim, dropout_rate) for _ in range(depth)
#         ]

#         self.decoder = tf.keras.Sequential([
#             layers.Dense(embed_dim),
#             layers.Reshape((img_size // patch_size, img_size // patch_size, embed_dim)),
#             layers.Conv2DTranspose(filters=embed_dim // 2, kernel_size=2, strides=2, activation="relu"),
#             layers.Conv2DTranspose(filters=embed_dim // 4, kernel_size=2, strides=2, activation="relu"),
#             layers.Conv2DTranspose(filters=embed_dim // 8, kernel_size=2, strides=2, activation="relu"),
#             layers.Conv2D(num_classes, kernel_size=1, activation="softmax")
#         ])


#     def call(self, inputs):
#         # Преобразование в патчи и добавление позиционной эмбеддинга
#         x = self.patch_embedding(inputs)
#         x += self.position_embedding

#         # Пропуск через трансформерные блоки
#         for block in self.transformer_blocks:
#             x = block(x)

#         # Декодер для восстановления пространственной размерности
#         x = self.decoder(x)
#         x = tf.image.resize(x, (self.img_size, self.img_size))  # Явно задаем размер выхода
#         return x

In [7]:
# Трансформерный блок
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim:int, num_heads:int, mlp_dim:int, dropout_rate=0.1):
        super().__init__()
        self.attention = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.dropout1 = layers.Dropout(dropout_rate)
        self.norm1 = layers.LayerNormalization(epsilon=1e-6)

        # многослойный персиптрон 
        self.mlp = tf.keras.Sequential([
            layers.Dense(mlp_dim, activation=tf.nn.gelu),
            layers.Dropout(dropout_rate),
            layers.Dense(embed_dim),
            layers.Dropout(dropout_rate),
        ])
        self.norm2 = layers.LayerNormalization(epsilon=1e-6)

    def call(self, inputs, training=False):
        attn_output = self.attention(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.norm1(inputs + attn_output)

        mlp_output = self.mlp(out1, training=training)
        return self.norm2(out1 + mlp_output)

In [9]:
class VisionTransformer(Model):
    def __init__(
        self, 
        img_size:int, 
        patch_size:int, 
        num_classes:int, 
        embed_dim:int, 
        depth:int, 
        num_heads:int, 
        mlp_dim:int, 
        dropout_rate=0.1
    ):
        super().__init__()
        self.img_size = img_size
        self.patch_size = patch_size
        self.num_patches = (img_size // patch_size) ** 2

        self.patch_embedding = PatchEmbedding(img_size, patch_size, embed_dim)
        self.position_embedding = self.add_weight(
            shape=(1, self.num_patches, embed_dim),
            initializer="random_normal",
            trainable=True
        )

        self.transformer_blocks = [
            TransformerBlock(embed_dim, num_heads, mlp_dim, dropout_rate) for _ in range(depth)
        ]

        self.decoder = tf.keras.Sequential([
            layers.Dense(embed_dim),
            layers.Reshape((img_size // patch_size, img_size // patch_size, embed_dim)),
            layers.Conv2DTranspose(filters=embed_dim // 2, kernel_size=2, strides=2, activation="relu"),
            layers.Conv2DTranspose(filters=embed_dim // 4, kernel_size=2, strides=2, activation="relu"),
            layers.Conv2DTranspose(filters=embed_dim // 8, kernel_size=2, strides=2, activation="relu"),
            layers.Conv2D(num_classes, kernel_size=1, activation="softmax")
        ])

    def call(self, inputs):
        # Преобразование в патчи и добавление позиционной эмбеддинга
        x = self.patch_embedding(inputs)
        x += self.position_embedding

        # Пропуск через трансформерные блоки
        for block in self.transformer_blocks:
            x = block(x)

        # Декодер для восстановления пространственной размерности
        x = self.decoder(x)
        x = tf.image.resize(x, (self.img_size, self.img_size))  # Явно задаем размер выхода
        return x

class PatchEmbedding(layers.Layer):
    def __init__(self, img_size:int, patch_size:int, embed_dim:int):
        super().__init__()
        self.img_size = img_size
        self.patch_size = patch_size
        self.projection = layers.Conv2D(
            embed_dim, kernel_size=patch_size, strides=patch_size, padding='valid'
        )

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        h, w = self.img_size, self.img_size
        if inputs.shape[1] != h or inputs.shape[2] != w:
            inputs = tf.image.resize(inputs, (h, w))

        x = self.projection(inputs)  # Apply Conv2D to project
        x = tf.reshape(x, (batch_size, -1, x.shape[-1]))  # Reshape to (batch_size, num_patches, embed_dim)
        return x


In [10]:
# # Параметры модели
PATCH_SIZE = 16 # Размер патча
EMBED_DIM = 512  # Увеличенная размерность эмбеддингов
DEPTH = 12  # Количество трансформерных блоков
NUM_HEADS = 16  # Голов для Multi-Head Attention
MLP_DIM = 1024  # Размерность скрытого слоя в MLP
DROPOUT_RATE = 0.2

PATCH_SIZE = 16  # Было 32
EMBED_DIM = 256  # Было 512
DEPTH = 6        # Было 12
NUM_HEADS = 8    # Было 16
MLP_DIM = 512    # Было 1024
DROPOUT_RATE = 0.2

In [11]:
# Создание модели
vit_segmentation_model = VisionTransformer(
    img_size=IMG_SIZE,
    patch_size=PATCH_SIZE,
    num_classes=NUM_CLASSES,
    embed_dim=EMBED_DIM,
    depth=DEPTH,
    num_heads=NUM_HEADS,
    mlp_dim=MLP_DIM,
    dropout_rate=DROPOUT_RATE
)

I0000 00:00:1734867962.609212   34081 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1734867962.609525   34081 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1734867962.609568   34081 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1734867962.791520   34081 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1734867962.791593   34081 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-12-22

In [12]:
# Компиляция модели
vit_segmentation_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)


In [13]:
# Вывод структуры модели
vit_segmentation_model.build((None, IMG_SIZE, IMG_SIZE, 3))
vit_segmentation_model.summary()


/home/midv/anaconda3/lib/python3.12/site-packages/keras/src/layers/layer.py:391: UserWarning: `build()` was called on layer 'vision_transformer', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


Model: "vision_transformer"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ patch_embedding                 │ ?                      │   0 (unbuilt) │
│ (PatchEmbedding)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block               │ ?                      │   0 (unbuilt) │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_1             │ ?                      │   0 (unbuilt) │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_2             │ ?                      │   0 (unbuilt) │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_3             │ ?                      │   0 (unbuilt) │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_4             │ ?                      │   0 (unbuilt) │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_5             │ ?                      │   0 (unbuilt) │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_6 (Sequential)       │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 262,144 (1.00 MB)

 Trainable params: 262,144 (1.00 MB)

 Non-trainable params: 0 (0.00 B)

In [14]:
# Сохранение схемы модели
plot_model(vit_segmentation_model, to_file="vit_model_architecture.png", show_shapes=True)

# Функции для оценки
from sklearn.metrics import confusion_matrix, f1_score, jaccard_score


In [15]:
def evaluate_model(y_true, y_pred, num_classes):
    y_true_flat = y_true.flatten()
    y_pred_flat = np.argmax(y_pred, axis=-1).flatten()

    cm = confusion_matrix(y_true_flat, y_pred_flat, labels=range(num_classes))
    iou = jaccard_score(y_true_flat, y_pred_flat, average="macro")
    f1 = f1_score(y_true_flat, y_pred_flat, average="macro")

    mean_accuracy = np.diag(cm).sum() / cm.sum()

    return mean_accuracy, iou, f1


In [16]:
# Генерация гистограммы результатов
def plot_metrics(metrics, labels):
    plt.bar(labels, metrics, color=['blue', 'green', 'red'])
    plt.xlabel("Метрики")
    plt.ylabel("Значения")
    plt.title("Оценка модели")
    plt.show()


In [17]:
# График обучения
class TrainingPlotCallback(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs=None):
        self.history = {"loss": [], "accuracy": []}

    def on_epoch_end(self, epoch, logs=None):
        self.history["loss"].append(logs.get("loss"))
        self.history["accuracy"].append(logs.get("accuracy"))
        
        plt.figure(figsize=(10, 5))
        plt.subplot(1, 2, 1)
        plt.plot(self.history["loss"], label="Loss")
        plt.title("Training Loss")
        plt.legend()

        plt.subplot(1, 2, 2)
        plt.plot(self.history["accuracy"], label="Accuracy")
        plt.title("Training Accuracy")
        plt.legend()

        plt.show()


In [18]:
# Функция для загрузки данных
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [19]:
# def process_path(image_path, mask_path):
#     img = load_img(image_path, target_size=(IMG_SIZE, IMG_SIZE))
#     mask = load_img(mask_path, target_size=(IMG_SIZE, IMG_SIZE), color_mode="grayscale")
#     img = img_to_array(img) / 255.0
#     mask = img_to_array(mask).astype("int")
#     return img, mask


In [20]:
# def process_path(image_path, mask_path):
#     img = load_img(image_path, target_size=(IMG_SIZE, IMG_SIZE))
#     mask = load_img(mask_path, target_size=(IMG_SIZE, IMG_SIZE), color_mode="grayscale")
#     img = img_to_array(img) / 255.0
#     mask = img_to_array(mask).astype("int")

#     # Удаляем лишнее измерение, если оно есть
#     # if len(mask.shape) == 3 and mask.shape[-1] == 1:
#     #     mask = tf.squeeze(mask, axis=-1)
#     return img, mask

# def data_generator(image_dir, mask_dir, limit=6000):
#     image_filenames = sorted(os.listdir(image_dir))[:limit]
#     mask_filenames = sorted(os.listdir(mask_dir))[:limit]
#     for img_filename, mask_filename in zip(image_filenames, mask_filenames):
#         img_path = os.path.join(image_dir, img_filename)
#         mask_path = os.path.join(mask_dir, mask_filename)
#         yield process_path(img_path, mask_path)

# def load_dataset(image_dir, mask_dir):
#     dataset = tf.data.Dataset.from_generator(
#         lambda: data_generator(image_dir, mask_dir),
#         output_types=(tf.float32, tf.int32),
#         output_shapes=((IMG_SIZE, IMG_SIZE, 3), (IMG_SIZE, IMG_SIZE, 1))
#     )
#     return dataset

In [21]:
def count_is_nune(train_data_dir: list, train_mask_dir: list) -> tuple[str, str]:
    count_mask = 0 
    count_image = 0 
    for img_path, mask_path in zip(train_data_dir, train_mask_dir):
        if not os.path.isfile(img_path):
            count_image += 1
            print(f"Файл изображения не найден: {img_path}")
        if not os.path.isfile(mask_path):
            count_mask += 1
            print(f"Файл маски не найден: {mask_path}")
    
    return f"Количество ошибок масок = {count_mask}", f"Количество ошибок изображений = {count_image}"

In [22]:

def parse_image_mask(image_path, mask_path):
    """Обрабатывает пары изображений и масок."""
    # Чтение изображения и маски
    img = tf.io.read_file(image_path)
    img = tf.image.decode_image(img, channels=3)
    img.set_shape([None, None, 3])
    img = tf.image.resize(img, [IMG_SIZE, IMG_SIZE])  # ошибка ? 
    img = tf.cast(img, tf.float32) / 255.0  # Нормализация

    mask = tf.io.read_file(mask_path)
    mask = tf.image.decode_image(mask, channels=1)
    mask.set_shape([None, None, 1])
    mask = tf.image.resize(mask, [IMG_SIZE, IMG_SIZE], method="nearest")  # Для масок используем nearest
    mask = tf.cast(mask, tf.int32)  # Целочисленные значения для меток

    print(img.shape)
    print(mask.shape)
    
    return img, mask

def load_dataset(image_dir, mask_dir, limit=None, batch_size=32, shuffle=True):
    """Загружает датасет из директорий изображений и масок."""
    # Получение путей к изображениям и маскам
    print("я в load_dataset")
    
    count_err = 0
    try:
        image_paths = sorted([os.path.join(image_dir, fname) for fname in os.listdir(image_dir)])
        mask_paths = sorted([os.path.join(mask_dir, fname) for fname in os.listdir(mask_dir)])
    except Exception as e:
        count_err += 1
        print(e)
        
    print(f"{count_err = }")
    
    if limit:
        image_paths = image_paths[:limit]
        mask_paths = mask_paths[:limit]
        
    a, b = count_is_nune(image_paths, mask_paths)
    
    print(a)
    print(b)
    
    # Создание TensorFlow Dataset
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, mask_paths))
    dataset = dataset.map(parse_image_mask, num_parallel_calls=tf.data.AUTOTUNE)
    
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(image_paths), reshuffle_each_iteration=True)

    dataset = dataset.batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE).prefetch(buffer_size=tf.data.AUTOTUNE)

    return dataset

In [23]:
import sys

train_data_dir = os.path.join(TRAIN_DIR, "images", "rgb")
train_mask_dir = os.path.join(TRAIN_DIR, "masks")

valid_data_dir = os.path.join(VAL_DIR, "images", "rgb")
valid_mask_dir = os.path.join(VAL_DIR, "masks")

if not os.path.isdir(DATASET_ROOT_DIR) \
    or not os.path.isdir(train_data_dir) \
    or not os.path.isdir(train_mask_dir) \
    or not os.path.isdir(valid_data_dir) \
    or not os.path.isdir(valid_mask_dir):
        sys.exit("Dataset dir is not exist")
else: 
    print("Ok")

Ok


In [24]:
# for img_path, mask_path in zip(valid_data_dir, valid_mask_dir):
#     if not os.path.isfile(img_path):
#         print(f"Файл изображения не найден: {img_path}")
#     if not os.path.isfile(mask_path):
#         print(f"Файл маски не найден: {mask_path}")

In [25]:
train_dataset = load_dataset(train_data_dir, train_mask_dir, limit=1000, batch_size=16)
val_dataset = load_dataset(valid_data_dir, valid_mask_dir, limit=1000, batch_size=16)

train_dataset = train_dataset.batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)


я в load_dataset
count_err = 0
Количество ошибок масок = 0
Количество ошибок изображений = 0
(512, 512, 3)
(512, 512, 1)
я в load_dataset
count_err = 0
Количество ошибок масок = 0
Количество ошибок изображений = 0
(512, 512, 3)
(512, 512, 1)


In [26]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Динамический вывод и сохранение модели
class TrainingPlotCallback(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs=None):
        self.history = {"loss": [], "val_loss": [], "accuracy": [], "val_accuracy": []}

    def on_epoch_end(self, epoch, logs=None):
        # Обновляем историю
        self.history["loss"].append(logs.get("loss"))
        self.history["val_loss"].append(logs.get("val_loss"))
        self.history["accuracy"].append(logs.get("accuracy"))
        self.history["val_accuracy"].append(logs.get("val_accuracy"))
        
        # График тренировки
        plt.figure(figsize=(12, 6))
        
        # Потери
        plt.subplot(1, 2, 1)
        plt.plot(self.history["loss"], label="Train Loss", marker='o')
        plt.plot(self.history["val_loss"], label="Val Loss", marker='o')
        plt.title("Loss")
        plt.legend()

        # Точность
        plt.subplot(1, 2, 2)
        plt.plot(self.history["accuracy"], label="Train Accuracy", marker='o')
        plt.plot(self.history["val_accuracy"], label="Val Accuracy", marker='o')
        plt.title("Accuracy")
        plt.legend()

        plt.show()



In [27]:
# Сохранение модели после каждой эпохи
checkpoint_callback = ModelCheckpoint(
    filepath="model_epoch_{epoch:02d}.keras",
    save_best_only=False,
    save_weights_only=False,
    verbose=1
)

In [28]:
# Обучение модели
history = vit_segmentation_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=50,
    callbacks=[TrainingPlotCallback(), checkpoint_callback]
)

# # Обучение модели
# history = vit_segmentation_model.fit(
#     train_dataset,
#     validation_data=val_dataset,
#     epochs=50,
#     callbacks=[TrainingPlotCallback()]
# )

Epoch 1/50


ValueError: Exception encountered when calling PatchEmbedding.call().

[1m'images' must have either 3 or 4 dimensions.[0m

Arguments received by PatchEmbedding.call():
  • inputs=tf.Tensor(shape=(None, None, 512, 512, 3), dtype=float32)

In [ ]:
# Пример использования метрик на валидационном наборе
val_images, val_masks = next(iter(val_dataset))
val_predictions = vit_segmentation_model.predict(val_images)
metrics = evaluate_model(val_masks.numpy(), val_predictions, NUM_CLASSES)
plot_metrics(metrics, ["Mean Accuracy", "IoU", "F1-Score"])

print("Обучение завершено. Схема модели сохранена в 'vit_model_architecture.png'")
